# import librairies

In [ ]:
import numpy as np
import scipy.io as sio
from scipy.stats import beta
import matplotlib.pyplot as plt
import seaborn as sns

# construct two classes of probability density functions (PDFs) from beta distributions

In [ ]:
a1, b1 = 2, 2 # parameters of class 1 
a2, b2 = 1.8, 2 # parameters of class 2

# define inputs (time instances)

In [ ]:
d=100 # fix the number of points per PDF
t = np.linspace(0,1, d) 

# simulate PDFs

In [ ]:
Nsimulations=500 # fix the number of observations per class
p1=np.zeros([Nsimulations,d]) # initialize the first class
p2=np.zeros([Nsimulations,d]) # initialize the second class
for i in range(Nsimulations):
    p1[i,:]=beta.pdf(t, a1+np.random.uniform(-0.2,0.2), b1) # PDFs of the first class
    p2[i,:]=beta.pdf(t, a2+np.random.uniform(-0.2,0.2), b2) # PDFs of the second class

# plot some examples of PDFs (the integral should be equal to one)

In [ ]:
sns.set_style("whitegrid")
plt.rcParams.update({'font.size': 25})
plt.rcParams.update({'legend.fontsize': 25})
plt.rcParams['font.weight'] = 'bold'

fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(111)

for j in range(50):
    plt.plot(t,p1[j,:],color='b',label="Class 1")
    plt.plot(t,p2[j,:],color='r',label="Class 2")
    
ax.legend(['Class 1', 'Class 2'], loc='best')

plt.show()

# find the corresponding square-root density functions (SRDFs) 

In [ ]:
psi1=np.sqrt(p1); psi2=np.sqrt(p2)

# project the SRDFs into the tangent space of the Hilbert sphere at the unity pole: $\mathbf{1}(t)=t$ using the $Log_\mathbf{1}(.)$ map

In [ ]:
def Log1(q): 
    x=np.linspace(0,1,len(q))
    """""if np.trapz(q*1,x)>1:
        z=0
    else:"""
    z=np.arccos(np.trapz(q*1,x))/np.sin(np.arccos(np.trapz(q*1,x)))*(q-(np.trapz(q,x))*np.ones(len(q)))
    return(z)

In [ ]:
g1=np.zeros([psi1.shape[0],psi1.shape[1]])
g2=np.zeros([psi2.shape[0],psi2.shape[1]])

for i in range(psi1.shape[0]):
    g1[i,:]=Log1(psi1[i,:])
    
for i in range(psi2.shape[0]):
    g2[i,:]=Log1(psi2[i,:]) 

# plot some examples of $Log_\mathbf{1}(\psi)$ (the integral should be equal to zero)

In [ ]:
sns.set_style("whitegrid")
plt.rcParams.update({'font.size': 25})
plt.rcParams.update({'legend.fontsize': 25})
plt.rcParams['font.weight'] = 'bold'

fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(111)

for j in range(50):
    plt.plot(t,g1[j,:],color='b',label="Class 1")
    plt.plot(t,g2[j,:],color='r',label="Class 2")
    
ax.legend(['Class 1', 'Class 2'], loc='best')

plt.show()

# concatenate all projected functions together

In [ ]:
g=np.concatenate((g1,g2)) 
np.shape(g)

# construct the binary vector of outputs 0/1 

In [ ]:
y = np.vstack((np.ones((p1.shape[0], 1)),np.zeros((p2.shape[0], 1))))
n=len(y) # check dimension
print(n)

# import the sklearn library 

In [ ]:
import sklearn as sk
from sklearn.metrics import accuracy_score, log_loss, roc_auc_score
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

# results of the GP classifier applied to projected functions

In [ ]:
import random

gp_opt = GaussianProcessClassifier(kernel=1.0 * RBF(length_scale=1.0)) # fix the class of kernel 

ind=random.sample(range(0, n), int(3*n/4))
index=np.isin(range(0, n), ind) # random split with 75% for training and 25% for test

xtrain=g[index] # training input 
ytrain=y[index] # training output
xtest=g[~index] # test input
ytest=y[~index] # test output
    
gp_opt.fit(xtrain, ytrain) # learn the GP classifier

NMLL=-gp_opt.log_marginal_likelihood(gp_opt.kernel_.theta) # negative marginal log-likelihood

ACC=accuracy_score(ytest, gp_opt.predict(xtest)) # accuracy score

AUC=roc_auc_score(ytest, gp_opt.predict_proba(xtest)[:, 1]) # AUC criteria
        
LOSS=log_loss(ytest, gp_opt.predict_proba(xtest)[:, 1]) # loss measure

# display the optimal kernel parameters

In [ ]:
print(gp_opt.kernel_) 

# plot the class 1 probability with test data together

In [ ]:
sns.set_style("whitegrid")
plt.rcParams.update({'font.size': 25})
plt.rcParams.update({'legend.fontsize': 25})
plt.rcParams['font.weight'] = 'bold'

fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(111)

plt.scatter(range(1,ytest.shape[0]+1) , ytest, s=50,color='g', marker=(5, 0), label="Test data")
plt.scatter(range(1,ytest.shape[0]+1),gp_opt.predict_proba(xtest)[:, 1],s=50,color='b', marker=(5, 0),label="Class 1 probability")
plt.plot([1, ytest.shape[0]],[0.5, 0.5],c="r")

plt.ylim(-0.2, 1.2)
plt.legend(loc="best")
plt.xticks([])
plt.show()